In [186]:
import sys
import rootpath
sys.path.append(rootpath.detect())

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

from testsuite.utilities import Pareto_split

from itertools import combinations

In [3]:
def same_side(V, a, b):
    """Test whether points are on the same side of a plane.
    
    The plane is defined by the points forming the _rows_ of V and the origin.
    
    Parameters
    ----------
    V : ndarray of shape (d-1, d)
        d-1 points that together with the origin define a plane 
        in d dimensions

    a : ndarray of shape (m,d)
    b : ndarray of shape (m,d)
        a[i] and b[i] are compared, for i = 1,...,m
        a and b must be 2d arrays
        
        
    Returns
    -------
    same : Bool ndarray of shape (m,)
        same[i] is True iff a[i] and b[i] are on the same side of the plane. 
    """
    assert len(a.shape) == 2 and len(b.shape) == 2
    assert a.shape == b.shape
    m, d = a.shape
    assert V.shape == (d-1, d)
    
    # Get the normal to the plane by via the SVD because the plane 
    # passes through the origin, so its distance to the origin is zero
    # The normal to the plane is the null space of the matrix of the points
    # defining the plane, including the orgin.
    
    A = np.vstack((V, np.zeros(d)))
    U, s, Vt = np.linalg.svd(A.T)
    assert s[-1] < 1e-10
    n = np.squeeze(U[:,-1])
    
    # They are on the same side if dot product with normal has the same sign
    same = np.sign(a@n) == np.sign(b@n)
    return same

In [379]:
def ssd_points_to_vector(V, points):
    assert points.shape[1] == V.shape[0]
    
    lenv = np.linalg.norm(points, axis=1)
    cs = vector_point_distances(V, points)
    S =  np.sqrt(1-cs*cs)*lenv
    return S.sum()

def vector_point_distances(V, points):
    assert points.shape[1] == V.shape[0]
    Vhat = V/np.linalg.norm(V)
    lenp = np.linalg.norm(points, axis=1)
    phat = (points.T/lenp).T
    cs = phat @ Vhat.T
    return cs

In [190]:
def in_cones(P, x):
    """
    Determine which cones formed from the origin and points in P
    enclose the vector x
    
    Parameters
    ----------
    
    P : ndarray, shape (m, d)
        Array of m d-dimensional points.  
        Cones are formed from combinations of d of these and the origin.
        
    x : ndarray, shape (d,)
        The ray to be enclosed. 
        
    Returns
    -------
    
    cones : list of d-dimensional tuples.
        Each tuple is the indices of the rows of P which, 
        with the origin, define the cone enclosing x.
        cones is empty if no combination of the points encloses x.
    """
    m, d = P.shape
    assert len(x.shape) == 1 and x.shape[0] == d
    
    X = np.tile(x, (m-(d-1), 1))  # Replicate x for same_side
    
    # For all possible d-dimensional planes defined by P and the origin, find 
    # whether x is on the same side as each of the remaining points in P. 
    # Store these in a dictionary whose key is a d-tuple of the point (first
    # element)and the indices of the points defining the plane.
    side = {}
    allpoints = set(range(m))
    for plane in combinations(allpoints, d-1):
        Iplane = list(plane)
        pts = allpoints.difference(set(plane))
        Ipts = list(pts)
        S = same_side(P[Iplane], P[Ipts], X)
        
        for p, same in zip(Ipts, S):
            if same:
                side[(p, *Iplane)] = 1

    # For all possible combinations of d points that, together with the origin
    # define a cone, find the ones that contain x.
    cones = []
    for simplex in combinations(allpoints, d):
        simplex = set(simplex)
        for i in simplex:
            plane = sorted(simplex.difference((i,)))
            try:
                _ = side[(i, *plane)]
            except:
                break
        else:
            # x was on the same side of all the planes
            cones.append(tuple(simplex))
    return cones
    


In [395]:
class FinleyInCones:
    def __init__(self, y, dmv):
        
        self.dmv = dmv/np.linalg.norm(dmv)
        self.y = y
        self.p = Pareto_split(y)[0]
        
        # find initial cones from p which enclose dmv 
        self.update_p()
        self.Ibest = self.get_best_cone()
        
        
    def update_p(self):
        self.p_in_y = self._get_p_inds(self.p, self.y)
        self.p_cone_inds = np.asarray(in_cones(self.p, self.dmv), dtype=int)
        self.y_cone_inds = self._convert_cone_inds_p_to_y(self.p_cone_inds)
        self.p_distances = vector_point_distances(dmv, self.p)
    
#         S = np.asarray([ ssd(self.dmv, self.p[list(cone)]) for cone in self.p_cone_inds])
#         try:
#             k = np.argmin(S)
#             self.Ibest = list(self.p_cone_inds[k])
#         except ValueError:
#             self.Ibest = np.argmin([ssd(self.dmv, pi.reshape(1,-1)) for pi in self.p])
        
    @staticmethod 
    def _get_p_inds(p, y):
        return {k: int(i) for k, i in zip(range(p.shape[0]),
                                          [np.where(np.all(y == pi, axis=1))[0] for pi in p])}
    
    def _convert_cone_inds_p_to_y(self, p_cones):
        y_cones = map(lambda x: tuple([self.p_in_y[xi] for xi in x]), p_cones)
        return np.array(list(y_cones), dtype=int)
    
    def update(self, y_new):
        
        len_new = np.linalg.norm(y_new)
        cs = y_new @ self.dmv/len_new
        dist_new = np.sqrt(1-cs*cs)*len_new
        
        self.dist = np.append(self.dist, dist_new)
        self.y = np.append(self.y, y_new[np.newaxis,:], axis=0)
        self.update_p()
        
    def get_best_cone(self):
        if not self.Ibest:  # No enclosing cone yet, so try all combinations
            print('No enclosing cone')
            cones = in_cones(self.p, self.dmv)
            if cones:
                S = np.asarray([ssd_points_to_vector(dmv, self.p[list(cone),:]) for cone in cones])
                k = np.argmin(S)
                self.Ibest = list(cones[k])
                return self.p[self.Ibest,:], S[k]
            else:
                return None, None
            
        D = self.p_distances[self.Ibest]   # Distances to dm for best cone
        best_ssd = D@D
        if dist > np.max(D):
            print("Won't improve")
            # v won't improve the criterion, so return the old one.
            return self.V[self.Ibest,:], best_ssd
                
        # Try all the combinations of d-1 vertices with v 
        M, d = self.V.shape
        for others in combinations(range(M-1), d-1):
            Iverts = list((M-1, *others))
            D = self.dist[Iverts]
            S = D@D
            if S < best_ssd and self.inside(Iverts):
                self.Ibest = Iverts.copy()
                best_ssd = S
    
        return self.V[self.Ibest,:], best_ssd
    

In [396]:
M = 10
np.random.seed(22)

y = np.abs(np.random.randn(M, 3))
dmv = np.asarray([1, 2, 1], dtype='float')

In [397]:
op = FinleyInCones(y, dmv)

In [398]:
print(op.y.shape)
print(op.p.shape)

(10, 3)
(4, 3)


In [399]:
a = op.p[op.p_cone_inds]
b = op.y[op.y_cone_inds]
np.testing.assert_array_almost_equal(a, b)

print(a-b)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [400]:
from itertools import cycle
%matplotlib qt

def colour_cycle():
    return cycle(['tab:blue', 'tab:orange', 'tab:green', 'tab:red',
                  'tab:purple', 'tab:brown', 'tab:pink',
                  'tab:gray', 'tab:olive', 'tab:cyan'])

p = op.p
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(y[:,0], y[:,1], y[:,2])
ax.scatter(p[:,0], p[:,1], p[:,2], c="C1", s=150, alpha=0.2)
for n, v in enumerate(y):
    ax.text(*v, f'{n}')


dm = op.dmv
ax.plot([0, dm[0]], [0, dm[1]], [0, dm[2]], 'r', lw=2)

colour = colour_cycle()
for I in op.p_cone_inds:
    I = list(I)
    Z = op.p[I]
    ax.add_collection3d(Poly3DCollection(Z, alpha=0.2, color=next(colour)))
    

winner = op.p[op.Ibest]
try:
    ax.plot(*np.vstack((winner[-1], winner)).T, c="m")
except ValueError:
    ax.scatter(*winner.T, s=120, c="m")

AttributeError: 'FinleyInCones' object has no attribute 'Ibest'

/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [394]:
op.y_distances

/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/lib/function_base.py:393: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/finley/miniconda3/envs/testsuite/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


array([0.91870916, 0.79534056, 0.9778051 , 0.57095235, 0.90242371,
       0.88103258, 0.91838787, 0.75009753, 0.85492177, 0.9205804 ])

In [307]:
op.dmv.shape

(3,)

In [308]:

op.y.shape

(10, 3)